In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils.np_utils import to_categorical 
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop,Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import Model,layers
from sklearn.metrics import classification_report,confusion_matrix



![](https://www.mayoclinic.org/-/media/kcms/gbs/patient-consumer/images/2014/10/30/15/17/mcdc7_brain_cancer-8col.jpg)

# Utilizacion de Inteligencia Artificial para el reconocimiento de tumores cerebrales

Los tumores de cerebro son generalmente evaluados inicialmente mediante Resonancia Magnetica, y confirmados posteriormente con biopsia. A continuacion tratare de crear un prototipo de Modelo de Inteligencia Artificial, para poder clasificar diferentes tipos de tumores adecuadamente. Los datos que utilizo son de Kaggle, y estan conformados por multiples Resonancias Magneticas en T1 y T2, esta misma tiene 4 categorias. Las categorias estan conformadas por gliomas, meningiomas, tumores de hipofisis, y cerebros sin patologias. 

En una primera parte utilizare una Red Neuronal Convolucional tipica para el modelo, en una segunda etapa voy a utilizar una tecnica que se esta popularizando, llamada TransferLearning. Una de las grandes limitaciones de la inteligencia artificial es el dificil acceso a una base de datos en buenas condiciones estructurales. La utilizacion de redes neuronales "pre-entrenadas", puede sortear este obstaculo y generalmente suele producir mejores resultados sobre todo en datasets de baja calidad o de baja cuantia. 

In [ ]:
train_path = "/kaggle/input/brain-tumor-classification-mri/Training/"
test_path="/kaggle/input/brain-tumor-classification-mri/Testing/"

Checkeamos que tenemos bien las direcciones de nuestras Imagenes. 

In [ ]:
os.listdir(test_path)

In [ ]:
os.listdir(train_path)

# **Miremos algunas de las Resonancias Magneticas**



# Adenoma de hipofisis

In [ ]:
para_cell = "../input/brain-tumor-classification-mri/Training/pituitary_tumor/p (102).jpg"
para_img= imread(para_cell)
plt.imshow(para_img)

# Un Glioma

In [ ]:
para_cell1 = "../input/brain-tumor-classification-mri/Training/glioma_tumor/gg (112).jpg"
para_img= imread(para_cell1)
plt.imshow(para_img)

# Una Resonancia sin patologia

In [ ]:
para_cell2 = "../input/brain-tumor-classification-mri/Training/no_tumor/image (21).jpg"
para_img= imread(para_cell2)
plt.imshow(para_img)

# Un Meningioma

In [ ]:
para_cell3 = "../input/brain-tumor-classification-mri/Training/meningioma_tumor/m (125).jpg"
para_img= imread(para_cell3)
plt.imshow(para_img)

# Hacemos un pre-procesamiento de las imagenes, y producimos artificialmente un aumento en el volumen de imagenes de entrenamiento, como resultado esto nos prosporcionara un modelo mas robusto.[](http://)

In [ ]:
image_gen = ImageDataGenerator( ImageDataGenerator(
        featurewise_center=False,  
        samplewise_center=False, 
        rescale=1. / 255,
        zca_whitening=False,  
        rotation_range=0.9,
        zoom_range = 0.9,
        width_shift_range=0,  
        height_shift_range=0,  
        horizontal_flip=True,  
        vertical_flip=False)  
                              )

In [ ]:
image_shape = (224,224,3)

In [ ]:
image_gen.flow_from_directory(train_path)

In [ ]:
image_gen.flow_from_directory(test_path)

# Programamos una Red Neuronal Convolucional. 

In [ ]:
model = Sequential()


model.add(Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (224,224,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters = 128, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters = 128, kernel_size = (2,2),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.2))


model.add(Conv2D(filters = 256, kernel_size = (2,2),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(1024, activation = "relu"))
model.add(Dropout(0.5))

model.add(Dense(4, activation = "softmax"))

optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])


In [ ]:
model.summary()

In [ ]:
batch_size = 16

In [ ]:
train_image_gen = image_gen.flow_from_directory(train_path,
                                               target_size=image_shape[:2],
                                                color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='categorical')

In [ ]:
test_image_gen = image_gen.flow_from_directory(test_path,
                                               target_size=image_shape[:2],
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='categorical',shuffle=False)

early_stop = EarlyStopping(monitor='val_loss',patience=2)

In [ ]:
early_stop = EarlyStopping(monitor='val_loss',patience=10)

In [ ]:
history = model.fit_generator(train_image_gen,epochs=20,
                              validation_data=test_image_gen,
                              callbacks=[early_stop])

# Evaluemos que tambien le ha ido a nuestro modelo.

In [ ]:
plt.plot(history.history["accuracy"],c = "red")
plt.plot(history.history["val_accuracy"],c = "black")
plt.title("Loss")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend(["train", "test"])
plt.rcParams["figure.figsize"] = (6,6)
plt.show()

In [ ]:
model.evaluate_generator(test_image_gen)

Como podemos ver, el modelo tiene un 64% de Accuracy. Con 4 categorias (Con el mismo numero dei magenes en cada una), si eligieramos al azar tendriamos un 25% chances de acertar. Esto es bueno pero no lo suficiente. 

# Podemos guardar nuestro modelo en un simple archivo, y utilizarlo luego facilmente para clasificar tumores.

In [ ]:
model.save("model_brainTumor.h5")
classifier = load_model('model_brainTumor.h5')

# Como pudimos ver, la red neuronal tal vez no funciono tan bien como hubiesemos querido. Utilizaremos otra tecnica para ver si podemos mejorar los resultados. 

> # TransferLearning

TransferLearning, es una tecnica en la cual tomamos redes neuronales "pre-entrenadas" y las utilizamos en nuestro propio DataSet, en este caso, en las resonancias magneticas de tumores cerebrales. La tecnica consiste solamente en tomar las capas inferiores de la misma, que sirven para el reconocimiento de patrones generales, como linea y curvas, mientras que retiramos la capa de neuronas mas superficiales y que detectan patrones mas especificos, y lo reemplazamos por capas programadas por nosotros mismos. Esto reduce bastante el tiempo de entrenamiento y generalmente mejora el valor predictivo del modelo. 

# Resnet50

Resnet50 es una arquitectura de Redes Convolucionales, con una estructura Residual, inspirada en las Celulas Piramidales de la corteza cerebral. Esta arquitectura utiliza conexiones en salto, es decir, ciertas nueronas no conectan con la capa siguiente, como lo harian en un modelo convencional sino que saltan 2 o 3 capas mas adelante. Este tipo de arquitectura le proporciona al modelo la capacidad de apilar un gran numeros de capas neuronales sin correr riesgo del famoso "Vanishing Gradients".

In [ ]:
from tensorflow.keras.applications import ResNet50

pretrained_model=ResNet50( input_shape=(224,224,3),
                                  include_top=False,
                                  weights='imagenet'
                                   )

for layer in pretrained_model.layers:
     layer.trainable = False

pretrained_model.summary()

In [ ]:
last_layer = pretrained_model.get_layer('conv5_block3_out')
print('last layer of rsnet50 : output shape: ', last_layer.output_shape)
last_output = last_layer.output

x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)                  
x = layers.Dense(4, activation='softmax')(x)

model_resnet = Model(pretrained_model.input, x) 

In [ ]:
model_resnet.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['acc'])

In [ ]:
history = model_resnet.fit_generator(train_image_gen,epochs=20,
                              validation_data=test_image_gen,)

In [ ]:
plt.plot(history.history["acc"],c = "green")
plt.plot(history.history["val_acc"],c = "blue")
plt.axis([0, 20, 0, 1])
plt.title("Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epochs")
plt.legend(["train", "test"])
plt.rcParams["figure.figsize"] = (10,10)
plt.show()

In [ ]:
model_resnet.evaluate_generator(test_image_gen)

**Como podemos ver, nuestro nuevo modelo tiene unas chances del casi 80% de predecir correctamente un tumor de craneo. Bastante superior al 64% del Modelo anterior, incluso con menor tiempo de entrenamiento. Esto podria demostrar a priori la superioridad del TransferLearning.**

# Limitaciones

> Como pudimos ver, con unas simples lineas de codigo podemos brindar una elegante solucion a lo que podria ser una posible falta de personal especializado en reconocimiento de imagenes. Aunque un 80% de certeza, no es aceptable en el ambito medico, con un mayor numero de imagenes y de mayor caldiad podriamos aumentar la certeza del modelo.
Esto no reemplazaria el rol del profesional especializado pero podria ser una herramienta util en el workflow, o para proporcionar conocimientos tecnico en areas rurales o sin acceso a este tipo de profesionales. 



Autor: Gabriel Cerono